# RESEARCH NOTEBOOK --> XGridT


In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas_ta as ta  # noqa: F401
from core.data_sources import CLOBDataSource


# Initialize the data source
clob = CLOBDataSource()

In [ ]:
# Define the parameters
CONNECTOR_NAME = "binance_perpetual"
TRADING_PAIR = "WLD-USDT"
INTERVAL = "1m"

In [ ]:
clob.load_candles_cache()
candles = next(candle for candle in clob.candles_cache.values()
               if candle.trading_pair == TRADING_PAIR and
               candle.connector_name == CONNECTOR_NAME and
               candle.interval == INTERVAL)

In [ ]:
import plotly.graph_objects as go

ema_short = 12
ema_medium = 100
ema_long = 300


df = candles.data.copy()
df.ta.ema(length=ema_short, append=True)
df.ta.ema(length=ema_medium, append=True)
df.ta.ema(length=ema_long, append=True)

short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

df.loc[long_condition, "signal"] = 1
df.loc[short_condition, "signal"] = -1


fig = candles.fig(width=1200)
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=df.index, y=df[ema_fast],
                         line=dict(color='white', width=2),
                         name='Fast EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_med],
                         line=dict(color='blue', width=2), 
                         name='Medium EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df[ema_slow],
                         line=dict(color='violet', width=2),
                         name='Slow EMA'))

fig.show()


In [ ]:
df

In [ ]:
# Generate signal


short_ema = df[f"EMA_{ema_short}"]
medium_ema = df[f"EMA_{ema_medium}"]
long_ema = df[f"EMA_{ema_long}"]
close = df["close"]


long_condition = (short_ema > medium_ema) & (medium_ema > long_ema) & (short_ema > long_ema)
short_condition = (short_ema < medium_ema) & (medium_ema < long_ema) & (short_ema < long_ema)

df["signal"] = 0
df.loc[long_condition, "signal"] = 1
df.loc[short_condition, "signal"] = -1

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02,
                    subplot_titles=('OHLC with BB', 'MACD', 'Signal'),
                    row_heights=[0.6, 0.2, 0.2])

# Add candlestick
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['open'],
                             high=df['high'],
                             low=df['low'],
                             close=df['close'],
                             name='Candlesticks'),
              row=1, col=1)


# Add EMAs
ema_fast = f'EMA_{ema_short}'
ema_med = f'EMA_{ema_medium}' 
ema_slow = f'EMA_{ema_long}'

fig.add_trace(go.Scatter(x=df.index, y=df[ema_fast],
                         line=dict(color='#00FF00', width=2),
                         name='Fast EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df[ema_med],
                         line=dict(color='#FFA500', width=2), 
                         name='Medium EMA'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df[ema_slow],
                         line=dict(color='#0000FF', width=2),
                         name='Slow EMA'), row=1, col=1)



# Add the signal line
fig.add_trace(go.Scatter(x=df.index, y=df['signal'],
                         mode='lines',
                         name='Signal',
                         line=dict(color="white")),
              row=3, col=1)

# Update layout for dark theme
fig.update_layout(
    title=f'{exchange} - {trading_pair} - {interval}',
    width=1500, height=1000,
    font=dict(color='#e1e1e1'),
    plot_bgcolor='#1e1e1e',
    paper_bgcolor='#1e1e1e',
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor='rgba(0,0,0,0)'),
    yaxis=dict(title='Price'),
    yaxis2=dict(title='MACD', showgrid=False),
    yaxis3=dict(title='Signal', showgrid=False),
    showlegend=False
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#323232', zeroline=False)

# Show the plot
fig.show()
